In [2]:
import findspark
findspark.init()

# Spark function
from pyspark import Row, StorageLevel
from pyspark.sql import SparkSession, DataFrame
from pyspark.sql.functions import pandas_udf, udf, explode, array, when
from pyspark.sql.types import IntegerType, StringType, ArrayType,BooleanType
from pyspark.sql.window import Window
import pyspark.sql.functions as F

# Python function
import re
import subprocess
import numpy as np
import pandas
import pyarrow
from functools import reduce 
import copy
import time

appname = input("appname, folder name : ")
folder_name = copy.deepcopy(appname) 

# Start for Spark Session
spark = SparkSession.builder.master("spark://master:7077")\
                        .appName(appname)\
                        .config("spark.driver.memory", "8G")\
                        .config("spark.driver.maxResultSize", "5G")\
                        .config("spark.executor.memory", "25G")\
                        .config("spark.memory.fraction", 0.2)\
                        .config("spark.sql.shuffle.partitions", 100)\
                        .config("spark.eventLog.enabled", "true")\
                        .config("spark.cleaner.periodicGC.interval", "15min")\
                        .getOrCreate()

appname, folder name : gatk


In [1]:
def preVCF(hdfs, spark):
    vcf = spark.sparkContext.textFile(hdfs)
    #header_contig = vcf.filter(lambda x : re.match("^#", x))
    col_name = vcf.filter(lambda x : x.startswith("#CHROM")).first().split("\t")
    vcf_data = vcf.filter(lambda x : re.match("[^#][^#]", x))\
                       .map(lambda x : x.split("\t"))\
                       .toDF(col_name)\

    vcf_data = vcf_data.select(F.col("FORMAT"))
    return vcf_data

In [3]:
hdfs = "hdfs://master:9000/raw_data/gvcf_20.g.vcf"
gatk_vcf = preVCF(hdfs, spark).coalesce(100).cache()
gatk_vcf.count()

In [6]:
gatk_vcf.dropDuplicates().toPandas()

,FORMAT
0,GT:AD:DP:GQ:MIN_DP:PL:SB
1,GT:DP:GQ:MIN_DP:PL
2,GT:AD:DP:GQ:PL:SB
3,GT:AD:DP:GQ:PGT:PID:PL:SB
4,GT:DP:GQ:MIN_DP:PGT:PID:PL:SB
5,GT:DP:GQ:MIN_DP:PL:SB
6,GT:AD:DP:GQ:MIN_DP:PGT:PID:PL:SB


* GT 제외 알파벳 순 sort